In [1]:
# Import Dependencies 
from flask import Flask, render_template, redirect 
from flask_pymongo import PyMongo
import scrape_mars
import os
import pymongo

# Create an instance of Flask app
app = Flask(__name__)

# Use flask_pymongo to set up mongo connection locally 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to a database
db = client.mars_app

#Drop collection if avaible to remove duplates 
#db.mars_info.drop()

@app.route("/")
def index(): 
    # Return template and data
    # Find data
    mars_info_data = db.mars.find_one()
    return render_template("index.html", mars_info=mars_info_data)

@app.route("/scrape")
def scrape():
    try:
        mars_news = scrape_mars.scrape_mars_news()
        mars_image = scrape_mars.scrape_mars_image()
        mars_facts = scrape_mars.scrape_mars_facts()
        mars_weather = scrape_mars.scrape_mars_weather()
        data = scrape_mars.scrape_mars_facts()
        mars_hemispheres = scrape_mars.scrape_mars_hemispheres()   

        # Create Mission to Mars dictionary, which will be imported into Mongo
        mars_info = {}   

        # Dictionary entry from MARS NEWS
        mars_info['news_title'] = mars_news[0]
        mars_info['news_paragraph'] = mars_news[1]
        # Dictionary entry from fetured image created in line 14
        mars_info['featured_image_url'] = mars_image 
        # Dictionary entry from Weather twitts 
        mars_info['weather_tweet'] = mars_weather
        # Dictionary entry from MARS FACTS
        mars_info['mars_facts'] = data
        mars_info['hiu'] = mars_hemispheres

        db.mars.update({}, mars_info, upsert=True)
        mars_info_data = db.mars.find_one()
    except:
        mars_info_data = db.mars.find_one() 

    return render_template("index.html", mars_info=mars_info_data)

if __name__ == "__main__": 
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2019 19:00:21] "GET / HTTP/1.1" 200 -


InSight sol 156 (2019-05-05) low -99.2ºC (-146.6ºF) high -18.1ºC (-0.6ºF)
winds from the SW at 4.7 m/s (10.5 mph) gusting to 13.8 m/s (30.8 mph)
pressure at 7.40 hPapic.twitter.com/FlGDeYg8hi


C:\Users\Sisay\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
127.0.0.1 - - [06/May/2019 19:01:28] "GET /scrape HTTP/1.1" 200 -
